# 🔰PyTorchでニューラルネットワーク基礎 #04 【画像分類・畳み込み層】

* Qiitaの記事と連動しています
* 利用データ：aiueo.npz
* [独立行政法人産業技術総合研究所のETL文字データベース](http://etlcdb.db.aist.go.jp/)を利用させていただきました。「あ・い・う・え・お」の５種類の画像を抽出しMNISTと対応させるべく28x28サイズに縮小したものを利用しています

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [2]:
#デバイスの選択
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print("Using Device Name:", device)

Using Device Name: cuda:0


In [3]:
# 精度を計算する関数
def accuracy(y, t):
    _, argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy

In [4]:
data = np.load("./aiueo.npz")
x = data["x"]
t = data["t"]

# torchテンソルに変換
x = torch.FloatTensor(x).to(device)
t = torch.LongTensor(t).to(device)

x, x_test, t, t_test = train_test_split(x,t, test_size=0.1, random_state=55)

In [5]:
x.shape, x_test.shape, t.shape

(torch.Size([712, 1, 28, 28]), torch.Size([80, 1, 28, 28]), torch.Size([712]))

## 確認ポイント
1. カーネルサイズと出力後の形状
2. out_channelsの効果
3. flatten
4. Linearのin_featuresの計算方法
5. 活性化関数の引数
   * nn.LeakyReLU()の場合は、\_\_init__ 内でオプションを指定
   * F.leaky_relu()の場合は、forward()で使うときに、 l_relu(h, negative_slope=0.01)と使用時にオプションを記入
     

In [6]:
##### モデル定義
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=5 ,kernel_size=5)
        self.act1 = nn.LeakyReLU(negative_slope=0.01)  # Leaky ReLU導入
        self.flat = nn.Flatten()
        self.fc1 =  nn.Linear(in_features=5*24*24, out_features=100)
        self.act2 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=100, out_features=5)
        
    def forward(self, x):
        h1 = self.cnn1(x)
        h2 = self.act1(h1)
        h = self.flat(h2)
        h = self.act2(self.fc1(h))
        y = self.fc2(h)
        return y, h1, h2

In [7]:
model = DNN()
model.to(device)

DNN(
  (cnn1): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1))
  (act1): LeakyReLU(negative_slope=0.01)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2880, out_features=100, bias=True)
  (act2): ReLU()
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)

In [8]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model.parameters())

In [9]:
# 機械学習
LOOP = 150
for epoch in range(LOOP):
    optimizer.zero_grad()
    y,h1,h2 = model(x)
    loss = criterion(y, t)
    acc  = accuracy(y,t)
    loss.backward()
    optimizer.step()
    if (epoch+1)%10 == 0:
        print(f"{epoch}: loss: {loss.item()}, acc: {acc}")  # バッチ全体での損失や精度

9: loss: 0.9390642046928406, acc: 0.8567415730337079
19: loss: 0.4787551760673523, acc: 0.8974719101123596
29: loss: 0.336257666349411, acc: 0.9297752808988764
39: loss: 0.24548858404159546, acc: 0.9564606741573034
49: loss: 0.18027959764003754, acc: 0.9676966292134831
59: loss: 0.13152575492858887, acc: 0.973314606741573
69: loss: 0.09494158625602722, acc: 0.9817415730337079
79: loss: 0.06776448339223862, acc: 0.9873595505617978
89: loss: 0.04831302911043167, acc: 0.9901685393258427
99: loss: 0.03475889563560486, acc: 0.9957865168539326
109: loss: 0.025496196001768112, acc: 0.9985955056179775
119: loss: 0.01916482113301754, acc: 1.0
129: loss: 0.0148230642080307, acc: 1.0
139: loss: 0.011767580173909664, acc: 1.0
149: loss: 0.009557913988828659, acc: 1.0


In [10]:
model.eval()
y_test,_,_ = model(x_test)
test_acc = accuracy(y_test, t_test)
print(f"精度:{test_acc}")

精度:0.95
